In [ ]:
from __future__ import print_function
import numpy as np
import random
import pandas as pd
from scipy import stats
import sys, os
import logging
import tensorflow as tf
from keras import layers
from keras import regularizers
from keras.models import Model
from keras.models import Sequential
from keras.layers import *
from keras.regularizers import l1,l2, L1L2
from sklearn.metrics.pairwise import cosine_similarity
import keras
import keras.utils.np_utils as kutils
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,Callback,ModelCheckpoint,ReduceLROnPlateau
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import datasets, linear_model

import itertools

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import math as m
import keras.backend as K
import sklearn

/tmp/ipykernel_8386/385959801.py:20: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr


In [ ]:
nb_classes = 4

In [ ]:
def indices_to_one_hot(data,nb_classes):

	targets = np.array(data).reshape(-1)

	return np.eye(nb_classes)[targets]

In [ ]:
def readData(input):
    # Read the data
    data = pd.read_csv(input, sep='\t', header=0, na_values='nan')

    # Convert SNP, pheno, and folds columns to numeric
    SNP = data.iloc[:, 4:].apply(pd.to_numeric, errors='coerce').values
    pheno = pd.to_numeric(data.iloc[:, 1], errors='coerce').values
    folds = pd.to_numeric(data.iloc[:, 0], errors='coerce').values

    # Check for missing values
    if data.isnull().values.any():
        print("Warning: Missing values found in the data.")

    # Initialize array to store one-hot encoded SNPs
    nb_classes = 4  # Adjust this according to the number of SNP categories
    arr = np.empty(shape=(SNP.shape[0], SNP.shape[1], nb_classes))

    # Perform one-hot encoding for each SNP row
    for i in range(SNP.shape[0]):
        arr[i] = indices_to_one_hot(pd.to_numeric(SNP[i], downcast='signed'), nb_classes)

    # Return the SNPs (one-hot encoded), phenotypes, and fold indices
    return arr, pheno, folds


In [ ]:
def resnet(input):

	inputs = Input(shape=(input.shape[1],nb_classes))


	x = Conv1D(10,4,padding='same',activation = 'linear',kernel_initializer = 'TruncatedNormal', kernel_regularizer=regularizers.l2(0.1),bias_regularizer = regularizers.l2(0.01))(inputs)

	x = Conv1D(10,20,padding='same',activation = 'linear', kernel_initializer = 'TruncatedNormal',kernel_regularizer=regularizers.l2(0.1),bias_regularizer = regularizers.l2(0.01))(x)

	x = Dropout(0.75)(x)

	shortcut = Conv1D(10,4,padding='same',activation = 'linear',kernel_initializer = 'TruncatedNormal', kernel_regularizer=regularizers.l2(0.1),bias_regularizer = regularizers.l2(0.01))(inputs)
	x = layers.add([shortcut,x])

	x = Conv1D(10,4,padding='same',activation = 'linear',kernel_initializer = 'TruncatedNormal', kernel_regularizer=regularizers.l2(0.1),bias_regularizer = regularizers.l2(0.01))(x)

	x = Dropout(0.75)(x)
	x = Flatten()(x)

	x = Dropout(0.75)(x)

	outputs = Dense(1,activation = isru,bias_regularizer = regularizers.l2(0.01),kernel_initializer = 'TruncatedNormal',name = 'out')(x)

	model = Model(inputs = inputs,outputs = outputs)
	model.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adam(lr=0.001),metrics=['mae'])

	return model

In [ ]:
def compile_saliency_function(model):

	inp = model.layers[0].input
	outp = model.layers[8].output
	max_outp = K.max(outp, axis=1)
	saliency = K.gradients(K.sum(max_outp), inp)

	return K.function([inp,K.learning_phase()], saliency)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_images_plot(saliency, outname):
    plt.figure(figsize=(15, 8), facecolor='w')

    plt.subplot(2, 1, 1)
    x = np.median(saliency, axis=-1)
    plt.plot(x, 'b.')
    line = sorted(x, reverse=True)[10]
    plt.axhline(y=line, color='b', linestyle='--')
    plt.ylabel('Saliency Value', fontsize=15)

    plt.subplot(2, 1, 2)
    plt.axhline(y=line, color='r', linestyle='--')
    plt.xlabel('SNPs', fontsize=15)

    plt.savefig(outname)  # Save the plot
    plt.show()  # Display the plot


In [ ]:
def get_saliency(testSNP,model):

	array= np.array([testSNP])
	saliency_fn = compile_saliency_function(model)
	saliency_out = saliency_fn([[y for y in array][0],1])
	saliency = saliency_out[0]
	saliency = saliency[::-1].transpose(1, 0, 2)
	output= np.abs(saliency).max(axis=-1)

	return output

In [ ]:
a= 0.03  #height

def isru(x):
    return  x/(K.sqrt(1+a*K.square(x)))

In [ ]:
def model_train(test,val,train,testPheno,valPheno,trainPheno,model_save,weights_save):

	batch_size = 250
	earlystop = 5
	epoch = 300
	early_stopping = EarlyStopping(monitor='val_mean_absolute_error', patience=earlystop)

	model = resnet(train)
	history = model.fit(train, trainPheno, batch_size=batch_size, epochs=epoch, validation_data=(val,valPheno),callbacks=[early_stopping],shuffle= True)

	model.save(model_save)
	model.save_weights(weights_save)

	pred = model.predict(test)
	pred.shape = (pred.shape[0],)
	corr = pearsonr(pred,testPheno)[0]

	return history,corr



In [ ]:
from tensorflow.keras.models import load_model


In [ ]:
def main(IMP_input,QA_input):

	IMP_corr=[]
	QA_corr = []

	imp_SNP,imp_pheno, folds = readData(IMP_input)
	QA_SNP,QA_pheno, folds = readData(QA_input)

	PHENOTYPE = imp_pheno

	for i in range(1,11):

		testIdx = np.where(folds == i)
		if i == 10:
			valIdx = np.where(folds == 1)
			trainIdx = np.intersect1d(np.where(folds != i),np.where(folds != 1))
		else:
			valIdx = np.where(folds == i+1)
			trainIdx = np.intersect1d(np.where(folds != i),np.where(folds != i+1))

		trainSNP, trainSNP_QA , trainPheno = imp_SNP[trainIdx], QA_SNP[trainIdx], PHENOTYPE[trainIdx]
		valSNP, valSNP_QA, valPheno = imp_SNP[valIdx],QA_SNP[valIdx], PHENOTYPE[valIdx]
		testSNP, testSNP_QA, testPheno = imp_SNP[testIdx],QA_SNP[testIdx], PHENOTYPE[testIdx]

		model_l = resnet(testSNP)
		model_l.summary()
		# Load the weights into the model
		model_l.load_weights('model/model' + str(i) + '.h5')
		print("saliency output will be called "+str(i))
		# Get saliency map for test SNP using the trained model
		saliency_output = get_saliency(testSNP_QA, model_l)

		outname = 'My'+str(i)
		show_images_plot(saliency_output,outname)

		history, corr = model_train(testSNP,valSNP,trainSNP,testPheno,valPheno,trainPheno,'out'+str(i)+'.txt','model_weights'+str(i)+'.h5')
		IMP_corr.append(float('%0.4f' % corr))

		history, corr = model_train(testSNP_QA,valSNP_QA,trainSNP_QA,testPheno,valPheno,trainPheno,'model_'+str(i)+'.txt','model_weights'+str(i)+'.h5')
		QA_corr.append(float('%0.4f' % corr))

	print ("Average PCC (imputed) from 10-fold cross validation: " + str(np.mean(IMP_corr)))
	print ("Average PCC (non-imputed) from 10-fold cross validation: " + str(np.mean(QA_corr)))

In [ ]:
def MY_GRAPH(IMP_input, QA_input):

    IMP_corr = []
    QA_corr = []

    # Read input data
    imp_SNP, imp_pheno, folds = readData(IMP_input)
    QA_SNP, QA_pheno, folds = readData(QA_input)

    PHENOTYPE = imp_pheno

    for i in range(1, 11):

        testIdx = np.where(folds == i)
        if i == 10:
            valIdx = np.where(folds == 1)
            trainIdx = np.intersect1d(np.where(folds != i), np.where(folds != 1))
        else:
            valIdx = np.where(folds == i+1)
            trainIdx = np.intersect1d(np.where(folds != i), np.where(folds != i+1))

        # Split into training, validation, and testing sets
        trainSNP, trainSNP_QA, trainPheno = imp_SNP[trainIdx], QA_SNP[trainIdx], PHENOTYPE[trainIdx]
        valSNP, valSNP_QA, valPheno = imp_SNP[valIdx], QA_SNP[valIdx], PHENOTYPE[valIdx]
        testSNP, testSNP_QA, testPheno = imp_SNP[testIdx], QA_SNP[testIdx], PHENOTYPE[testIdx]

        model_l = resnet(testSNP)
        model_l.summary()
# Load the weights into the model
        model_l.load_weights('model/model' + str(i) + '.h5')
        print("saliency output will be called "+str(i))
        # Get saliency map for test SNP using the trained model
        saliency_output = get_saliency(testSNP, model_l)

        outname = 'My'+str(i)
        show_images_plot(saliency_output,outname)



        # Visualize saliency for each fold
#         plt.figure(figsize=(10, 6))
#         plt.imshow(saliency_output, cmap='hot', aspect='auto')
#         plt.colorbar()
#         plt.title("Saliency Map for Fold {}".format(i))
#         plt.xlabel('SNP Features')
#         plt.ylabel('Samples')
#         plt.savefig('saliency_map_fold_{}.png'.format(i))  # Save the plot
#         plt.show()  # Display the plot
#         plt.pause(0.001)  # Pause to allow the plot to render


# #         Optionally, save or print the saliency output
#         print("Saliency output for fold {}:".format(i))
#         print(saliency_output)

In [ ]:
import os
print(os.listdir('.'))  # This will show all files in the current directory


['QA_Our_model_weights1.h5', 'My 8.png', 'model_weights1.h5', 'QA_Our_model_9.txt', 'model_2.txt', 'QA_Our_model_5.txt', 'our10.txt', 'QA_Our_model_weights5.h5', 'out8.txt', 'model', 'QA_Our_model_weights8.h5', 'model_1.txt', 'My 7.png', 'model_weights4.h5', 'QA_Our_model_7.txt', 'model_3.txt', 'QA_Our_model_weights9.h5', 'our6.txt', 'our3.txt', 'My5.png', 'QA_Our_model_3.txt', 'Our_model_weights8.h5', 'My 10.png', 'Our_model_weights5.h5', 'our2.txt', 'our1.txt', 'our9.txt', 'model_weights9.h5', 'Our_model_weights1.h5', 'model_weights3.h5', 'out10.txt', 'My 9.png', 'My4.png', 'model_10.txt', 'My 4.png', 'My8.png', 'Our_model_weights7.h5', 'model_weights7.h5', 'model_4.txt', 'polytest.txt', 'model_weights6.h5', 'our7.txt', 'model_weights2.h5', 'My1.png', 'model_6.txt', 'My 3.png', 'IMP_height.txt', 'model_8.txt', 'out6.txt', 'QA_Our_model_2.txt', 'QA_Our_model_weights2.h5', 'My 6.png', 'QA_Our_model_4.txt', 'My 2.png', 'Our_model_weights6.h5', 'model_5.txt', 'My3.png', 'model_weights8.h

In [ ]:

if __name__ == '__main__':
    # os.chdir("MOISTURE")

    IMP_input = "IMP_height.txt"
   # QA_input = "QA_height.txt"
    print("Result for trained model")
    main(IMP_input,QA_input)

    #MY_GRAPH(IMP_input, QA_input)
    # Call the MY_GRAPH function



Result for trained model


AttributeError: 'DataFrame' object has no attribute 'convert_objects'